# 라인 메신저 Notify 알림 보내기

- line notify? 라인의 메시지 수신 전용 API
- 메시지 수신 기능만 제공하기 때문에, 대화형 챗봇/UX구성과 같은 복잡한 기능 활용은 라인 봇 활용    
- 계정별로 notify 토큰 받고 시작하자 : https://engineering.linecorp.com/ko/blog/line-notify-with-node-js-python-1-basic/

In [0]:
import os
import sys
import json

import requests
import urllib.request
import urllib
from bs4 import BeautifulSoup

# 라인 메신저 토큰
ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN_HERE'

# 네이버 개발자 센터 : 단축 URL 생성용
client_id = "YOUR_CLIENT_ID_HERE"
client_secret = "YOUR_CLIENT_SECRET_HERE"

### 테스트 과제  : 웹 크롤링 결과를 라인 notify로 보내기

- 타겟사이트 : 오늘의 유머 베오베 사이트
- 링크를 예쁘게 보이기 위해 단축URL 활용 : 네이버API(하루 2.5만건 무료 제공)

In [0]:
def send_linemsg(msg):
    URL = 'https://notify-api.line.me/api/notify'
    MESSAGE_FIELD = {'message' : msg}
    LINE_HEADERS = {'Authorization': 'Bearer ' + ACCESS_TOKEN}

    try:
          response = requests.post(
              url=URL,
              headers=LINE_HEADERS,
              data=MESSAGE_FIELD
          )
            print('Response HTTP Status Code: {status_code}'.format(
              status_code=response.status_code))
    except requests.exceptions.RequestException:
            print('HTTP Request failed')      


def make_short_url(url_origin):
    encText = urllib.parse.quote(url_origin)
    data = "url=" + encText

    url = "https://openapi.naver.com/v1/util/shorturl"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()          
    else:
        print("Error Code:" + rescode)

    return json.loads(response_body)['result']['url']    

In [0]:
url = 'http://www.todayhumor.co.kr/board/list.php?table=bestofbest'

req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

links = soup.select('td.subject > a')

msg = '오늘의 오유 | 베오베 \n'
for i, link in enumerate(links):
    title = str(i+1) + '. ' + link.text.strip()
    link_url = make_short_url('http://www.todayhumor.co.kr' + link.get('href'))
    msg = '\n'.join([msg,title,link_url,'\n'])
        if i > 18:         # 20개만 보내자 
    break

In [4]:
print(msg)

오늘의 오유 | 베오베 

1. 펌] 유부남이 말하는 결혼의 장점
http://me2.do/FCGO8W7w


2. 양궁 세계 신기록 기록 갱신 실패
http://me2.do/FSNJ7hDh


3. 행복한 고슴도치의 여행.jpg
http://me2.do/xhOeCRcY


4. 마약법 위반으로 체포된 앵무새.jpg
http://me2.do/5pOLsih6


5. [장문]  논란이 된, 비혼주의자 여성유튜버에게 달린 댓글.jpg
http://me2.do/GnTMy3o4


6. 남자 소변 자세 논란 종결.jpg
http://me2.do/xY6KsAdO


7. 아마추어들의 파티에 나타난 준프로
http://me2.do/I5Ssardf


8. 고대 로마제국의 의학수준ㄷㄷㄷ.JPG
http://me2.do/GOyWeL3e


9. 목 떨어져서 얼른 주워들음....
http://me2.do/x61fytkV


10. 운전법규를 잘 모르는 운전자
http://me2.do/5BSsjpDv


11. 심심풀이로 볼만한 움짤들 모음 609.GIF
http://me2.do/GRyn4Hl6


12. 직업 만족도 100% 카메라맨
http://me2.do/xLR2A0rO


13. 드라마 고등학생 역할 레전드.jpg
http://me2.do/5xiDpEGa


14. 러시아의 전통 코스튬.jpg
http://me2.do/G4yPo1T5


15. 중국에서 택시를 조심해야 하는 이유
http://me2.do/xQr72Myp


16. 칸쵸 코스프레
http://me2.do/FviVgmAK


17. 대왕 치즈돈까스 만드는 방법.jpg
http://me2.do/GdbUmzvj


18. 길가다 본 치명적 뒷태
http://me2.do/5tsYtFxv


19. 외래종유입이 무서운 이유
http://me2.do/596XoDZN


20. 새로산 드레스를 맘에 들어하는 딸을 보며 흐믓해하는 엄마의 구매후기
http://me2.do/FLJqW9PF




In [5]:
send_linemsg(msg)

Response HTTP Status Code: 200


심심하니 하나 더 하고 마무리. 웹사이트가 유사한 구조인 클리앙-모두의 공원

In [0]:
url = 'https://www.clien.net/service/board/park?od=T33'

req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
links = soup.select('div.list_title > a.list_subject')

msg = '클리앙 | 모두의 공원 \n'
for i, link in enumerate(links):
    title = str(i) + '. ' + link.text.strip()
    link_url = make_short_url('https://www.clien.net' + link.get('href'))
    if i == 0:           # 첫 게시글은 공지사항. 생략하자
        pass
    elif i > 18:         # 20개만 보내자 
        break 
    else:
        msg = '\n'.join([msg,title,link_url,'\n'])

In [7]:
print(msg)

클리앙 | 모두의 공원 

1. MBC가 시청률 일을 냈군요..
http://me2.do/xz3Ym8DC


2. 극혐)박찬주 아내 갑질 만행 정리
http://me2.do/FZKE7Dpd


3. 두달만에 60억이 600만원 됐네요....
http://me2.do/xXNs5Kkk


4. 진짜 욕 한 마디 하겠습니다 너무 화나네요
http://me2.do/xpDdAteG


5. 계엄령 문건 덮었다는 '윤석열 직인'···알고보니 자동 출력
http://me2.do/5yrznSUP


6. 유시민 이사장이 진짜 고수인거 같아요
http://me2.do/F4eGM1KV


7. 아름다운 대학생
http://me2.do/x2ElQBMX


8. 민경욱, 文대통령 모친상에 '모시고 살 수 없었나' 발언 논란
http://me2.do/5bIEs6s8


9. 한국기자협회 근황
http://me2.do/GIjZJpJd


10. 살인마들의 공통점.jpg
http://me2.do/GGNytAtc


11. 대통령 어머님이신걸 병원에서 몰랐다네요..
http://me2.do/5fSZp1xJ


12. MBN노조 '사과의 말씀 드리지만 종편 취소로 고용 위협되어선 일자리 정부 아닐것'
http://me2.do/GvrxTmtl


13. 검찰 외통수 걸렸군요. 유시민 이사장에게..
http://me2.do/xgUjSQEV


14. 문재인 케어 실감해보니 정말 고맙네요..
http://me2.do/FLJqTZvC


15. [자료] 리얼미터 대통령 48.5% 민주당 39.9%
http://me2.do/xtFLxjOS


16. (펌) 버클리대 석좌교수의 문재인 대통령 평가
http://me2.do/GNnGkxLF


17. 오늘 뉴스공장을 듣고 느낀 점 3가지
http://me2.do/xZmPbcfx


18. 프란치스코 교황, 문재인 대통령에 위로 메시지
http://me2.do/GvrxTmyK




In [8]:
send_linemsg(msg)

Response HTTP Status Code: 200
